In [4]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
from scipy.spatial.distance import pdist,squareform

In [5]:
def read_image(path):
    with Image.open(path + '.png') as img:
        width, height = img.size
        img = img.load()
        spatial_infor = []
        color_infor = []        
        print(f"read image {path}.png ")
        print(f"image width:{width} height:{height}")
        for i in range(height):
            for j in range(width):
                spatial_infor.append([i,j])
                color_infor.append(list(img[i,j]))
            
        return (spatial_infor, color_infor, (width, height))

def initial(image, gram, method, k):
    _, _, size = image
    width, height = size
    if method == "random":
        center = []
        center_x = np.random.randint(0,width,k)
        center_y = np.random.randint(0,height,k)
        for i in range(k):
            center.append([center_x[i],center_y[i]])    
        #print(center)
    elif method == "kmean++":
        center = []
        center_x = np.random.randint(0,width,1)
        center_y = np.random.randint(0,height,1)
        center.append([center_x[0],center_y[0]])
        for i in range(1,k):
            dist = []
            for index in range(height*width):
                dist.append(np.min([gram[c[0]*width+c[1]][index] for c in center]))
            probs = dist / sum(dist)
            cumprods = np.cumsum(probs)
            rand = np.random.rand()
            for j, p in enumerate(cumprods):
                if rand < p:
                    center.append([j//width, j%width])
                    break

    cluster = np.zeros((width * height, k))
    return center, cluster

def kernel_function(gamma_s, gamma_c, spatial_infor, color_infor):
    spatial_part = RBF_kernel(gamma_s, spatial_infor) 
    color_part = RBF_kernel(gamma_c, color_infor)
    gram = spatial_part * color_part
    return np.array(gram)

def RBF_kernel(gamma,X):
    kernel = np.exp( -1 * gamma * squareform(pdist(X,metric="euclidean")))
    return kernel

def dist_first(gram, center, k, height, width):
    dist = np.zeros((height * width, k))
    for i in range(height):
        for j in range(width):
            x1 = i * width + j
            for k_ in range(k):
                x2 = center[k_][0] * width + center[k_][1]
                #print(x1,x2)
                dist[x1][k_] = gram[x1][x2]
                if x1==x2:
                    dist[x1][k_] = 0
                #print(dist)
    #print(dist.shape)
    return dist

def dist_other(gram, cluster):
    # 2/|C| * Gram @ Center + 1/|C|^2 * @ eye @ Center.T @ Gram @ Center
    num_cluster = np.sum(cluster, axis=0, keepdims=1)
    #first_term is all the same so we ignore
    second_term = -2 * (gram @ cluster) / num_cluster
    third_term = np.ones(cluster.shape) @ ((cluster.T @ gram @ cluster )
     *np.eye(cluster.shape[1])) / (num_cluster**2)
    #print(second_term + third_term)
    return second_term + third_term

def visualization(image_path, k, init, method, type, cluster_record, spatial_infor, height, width):
    Blue = (102, 153, 255)
    Green = (102, 255, 153)
    Orange = (255, 187, 153)
    Red = (255, 153, 153)
    Yellow = (255, 255, 153)
    Purple = (255, 153, 255)
    color_list = [Blue, Green, Orange, Red, Yellow, Purple]
    
    image_record = []
    if method == None:
        save_path = "./visualization/" + type + "/" + init + image_path[1:] + "/" + str(k) + "/"
    else:
        save_path = "./visualization/" + type + "/" + init + "/" + method + image_path[1:] + "/" + str(k) + "/"
    print(f"save image and gif at {save_path}")

    for img_idx, cluster in enumerate(cluster_record):
        img = Image.new('RGB', (width, height), (255, 255, 255))
        for i in range(height * width):
            img.putpixel(spatial_infor[i], color_list[np.argmax(cluster[i])])
                # pass
        
        image_record.append(img)
        #img.show()
        img.save(save_path + image_path[1:] + "_" + str(k) + "_" + str(img_idx)+".png")

    img.save(save_path + image_path[1:] + "_" + str(k) + ".gif", save_all=True,  append_images = image_record)


#@nb.jit
def kernel_k_mean(image_path, init, k, epoch):
    image = read_image(image_path)
    spatial_infor, color_infor, size = image 
    width, height = size
    gamma_s, gamma_c = 0.001, 0.001
    gram = kernel_function(gamma_s, gamma_c, spatial_infor, color_infor)
    
    center, cluster = initial(image, gram, init, k) #cluster = alpha kn
    
    cluster_record = []

    for iter in range(epoch):
        # E-step
        if iter == 0:
            dist = dist_first(gram , center, k, height, width)             
        else:
            dist = dist_other(gram, cluster)

        # M-step
        new_cluster = np.zeros(cluster.shape)
        new_cluster[np.arange(height * width), np.argmin(dist,axis=1)] = 1
        num_cluster = np.sum(new_cluster,axis=0, keepdims=1)
        new_center = np.rint(new_cluster.T @ spatial_infor / num_cluster.T)

        cluster_record.append(new_cluster)
        
        cluster = new_cluster
        center = new_center

    visualization(image_path, k, init, None, "kernel_k_mean", cluster_record, spatial_infor, height, width)

# kernel_k_mean("./image1", "random", 3, 15)
# print("")
# kernel_k_mean("./image1", "kmean++", 3, 15)

# init_list = ["random", "kmean++"]
# for init in init_list:
#     for k in range(2,7):
#         kernel_k_mean("./image1", init, k, 15)
#         kernel_k_mean("./image2", init, k, 15)

# kernel_k_mean("./image2", "random", 3, 15)

In [6]:
def eigen_decomposition(similar, method):
    D = np.diag(np.sum(similar, axis=1))
    L = D - similar
    if method == "normalize":
        neg_sqrt_D = np.linalg.inv(D ** (1/2))
        L_sym = neg_sqrt_D @ L @ neg_sqrt_D
        eigenvalues, eigenvectors = np.linalg.eig(L_sym)
        return eigenvalues, eigenvectors
    elif method == "ratio":
        return np.linalg.eig(L)

def get_k_dimensional_euclidean_space(similar, k, method):
    eigenvalues, eigenvectors = eigen_decomposition(similar, method)
    index = np.argsort(eigenvalues)[1:k+1]
    return eigenvalues[index], eigenvectors[:,index]

def spectral_clustering(image_path, init, method, k, epoch):
    image = read_image(image_path)
    spatial_infor, color_infor, size = image 
    width, height = size
    gamma_s, gamma_c = 0.001, 0.001
    gram = kernel_function(gamma_s, gamma_c, spatial_infor, color_infor)
    
    #L = D - A  L:laplacian D:degree A:adjacency
    # A = gram
    k_dim_eigenvalues, k_dim_eigenvectors = get_k_dimensional_euclidean_space(gram, k, method)

    similarity = RBF_kernel(0.001, k_dim_eigenvectors)

    center, cluster = initial(image, similarity, init, k) #cluster = alpha kn
    
    cluster_record = []

    for iter in range(epoch):
        # E-step
        if iter == 0:
            dist = dist_first(similarity , center, k, height, width)             
        else:
            dist = dist_other(similarity, cluster)

        # M-step
        new_cluster = np.zeros(cluster.shape)
        new_cluster[np.arange(height * width), np.argmin(dist,axis=1)] = 1
        num_cluster = np.sum(new_cluster,axis=0, keepdims=1)
        new_center = np.rint(new_cluster.T @ spatial_infor / num_cluster.T)

        cluster_record.append(new_cluster)
        
        cluster = new_cluster
        center = new_center

    visualization(image_path, k, init, method, "spectral_clustering", cluster_record, spatial_infor, height, width)

#spectral_clustering("./image1" ,"random", "normalize", 3, 15)
# method_list = ["normalize", "ratio"]
# initial_list = ["random", "kmean++"]
# for method in method_list: 
#     for init in initial_list:
#         for k in range(2,7):
#             spectral_clustering("./image1" ,init, method, k, 15)
#             spectral_clustering("./image2" ,init, method, k, 15)